In [4]:
# Ячейка для очистки метаданных перед сохранением
from IPython.display import Javascript

def clean_metadata():
    return Javascript('''
        IPython.notebook.metadata.widgets = undefined;
        IPython.notebook.save();
    ''')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 14.8 MB/s eta 0:00:00


In [ ]:
pip install emoji

In [2]:
!pip install -q bitsandbytes
!pip install -q accelerate transformers peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 30.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.3 MB/s eta 0:00:00:00:0100:01


In [ ]:
pip install evaluate

In [16]:
from collections import Counter
from tqdm.auto import tqdm
from collections import defaultdict
import re, json, torch, os
import emoji
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModelForSequenceClassification, TrainingArguments, Trainer
import pandas as pd
import torch
from sklearn.utils.class_weight import compute_class_weight
from datasets import Dataset
import evaluate

In [14]:
import numpy as np

In [ ]:
df_train = pd.read_csv('/kaggle/input/thank-sirius/train.csv')
df_test = pd.read_csv('/kaggle/input/thank-sirius/test.csv')

In [ ]:
df_train.head(5)

,text
0,"Заказали 14.10.2017 , получили 25.10.2017 \r\n..."
1,"футболка хорошего качества,но футболка не как ..."
2,Все отлично!!!
3,"Рисунок не очень чёткий, а ткань прозрачная, в..."
4,плохо!!!Низ рваный..деньги не вернули!Открыла ...


# Проведу быстрый обзор, и покажу статистики

In [ ]:
def quick_eda(df, text_col='text', n_samples=5):
  print('Row, cols:', df.shape)
  print('Missing text:', df[text_col].isna().sum())

  df['_len_words'] = df[text_col].fillna('').str.split().str.len()
  df['_len_chars'] = df[text_col].fillna('').str.len()

  print('\nTop 5 shortest examples:')
  display(df.nsmallest(5, '_len_words')[[text_col, '_len_words']])
  print('\nTop 5 longest examples:')
  display(df.nlargest(5, '_len_words')[[text_col, '_len_words']])

  dup = df.duplicated(subset=[text_col]).sum()
  print(f'\nDuplicated text rows: {dup}')

  toks = Counter()
  for t in df[text_col].dropna().astype(str).sample(min(2000, len(df))):
    toks.update(t.lower().split())
  print('\nTop tokens (sampled):', toks.most_common(30)[:30])

In [ ]:
quick_eda(df_train)

Row, cols: (1818, 1)
Missing text: 0

Top 5 shortest examples:


,text,_len_words
102,Отлично!,1
305,Рекомендую.,1
395,Гавно,1
488,огонь!,1
567,Спасибо.,1



Top 5 longest examples:


,text,_len_words
697,"Брюки классные, полоса из кожзама, но мне прис...",41
17,"Товар так и не пришел. Заказала 6 июля, сейчас...",40
740,"ждала 60 с лишним дней, открыла спор чтобы хот...",40
26,"Я думала, что свитер будет более толстым , а н...",39
644,за 300 р. норм халат! на об 109 ог 109 хххл в ...,39



Duplicated text rows: 0

Top tokens (sampled): [('не', 1238), ('на', 795), ('и', 541), ('очень', 452), ('в', 398), ('но', 305), ('размер', 299), ('как', 194), ('качество', 187), ('товар', 187), ('по', 183), ('деньги', 175), ('с', 160), ('так', 155), ('за', 155), ('продавец', 144), ('что', 135), ('а', 133), ('все', 127), ('заказ', 121), ('я', 119), ('доставка', 106), ('ткань', 106), ('к', 101), (',', 99), ('соответствует', 96), ('рост', 87), ('платье', 85), ('заказала', 85), ('пришла', 76)]


*Top-токены показывают «размер», «качество», «доставка», «продавец» → видно, что будет много случаев «нет товара» (например, про доставку/споры).*

Минимальная очистка текста

In [ ]:
URL_RE = re.compile(r'https?://\S+')
HTML_RE = re.compile(r'<[^>]+>')
EMAIL_RE = re.compile(r'\S+@\S+')
PHONE_RE = re.compile(r'\+?\d[\d\-\s]{5,}\d')

def clean_text(t: str) -> str:
  if pd.isna(t):
    return ''
  t = str(t)
  t = t.strip()
  t = URL_RE.sub(' ', t)
  t = HTML_RE.sub(' ', t)
  t = EMAIL_RE.sub(' ', t)
  t = PHONE_RE.sub(' ', t)
  t = emoji.replace_emoji(t, replace=' ')
  t = re.sub(r'\s+', ' ', t)
  return t

In [ ]:
df_train['text_clean'] = df_train['text'].apply(clean_text)

In [ ]:
df_train.head(10)

,text,_len_words,_len_chars,text_clean
0,"Заказали 14.10.2017 , получили 25.10.2017 \r\n...",31,205,"Заказали 14.10.2017 , получили 25.10.2017 На м..."
1,"футболка хорошего качества,но футболка не как ...",34,203,"футболка хорошего качества,но футболка не как ..."
2,Все отлично!!!,2,14,Все отлично!!!
3,"Рисунок не очень чёткий, а ткань прозрачная, в...",10,63,"Рисунок не очень чёткий, а ткань прозрачная, в..."
4,плохо!!!Низ рваный..деньги не вернули!Открыла ...,9,75,плохо!!!Низ рваный..деньги не вернули!Открыла ...
5,"обычная майка с ужасным запахом, есть косяки н...",12,72,"обычная майка с ужасным запахом, есть косяки н..."
6,"Все как на фото, и цвет и форма.",8,32,"Все как на фото, и цвет и форма."
7,"Но рост 180 по колено,качество хорошее,доставк...",7,55,"Но рост 180 по колено,качество хорошее,доставк..."
8,Не подошло по размеру. Ткань тоже не понравилась,8,48,Не подошло по размеру. Ткань тоже не понравилась
9,"очень крутой топ,заказала 11.11 пришел 28.11 е...",10,64,"очень крутой топ,заказала 11.11 пришел 28.11 е..."


In [ ]:
kw = {
    "бытовая техника": [
        "пылесос", "утюг", "чайник", "микроволнов", "холодильник", "стирал", "посудомойк",
        "фен", "блендер", "тостер", "соковыжимк", "кондиционер", "обогревател", "пароварк"
    ],
    "обувь": [
        "кроссовк", "ботинк", "туфл", "сапог", "кеды", "балетк", "сандал",
        "мокасин", "сланц", "тапочк", "сабо"
    ],
    "одежда": [
        "футболк", "платье", "брюк", "свитер", "худи", "куртк", "рубашк", "майк", "шорты", "толстовк", "топ", "кофт", "джемпер",
        "шапк", "перчатк", "шарф", "носок", "жилет"
    ],
    "посуда": [
        "кастрюл", "тарелк", "стакан", "чашк", "ложк", "вилка", "сковород",
        "миск", "кружк", "ковш", "сито", "термос", "чайник"
    ],
    "текстиль": [
        "постель", "полотенц", "штор", "простын", "плед", "покрывало", "подушк", "одеяло",
        "наволочк", "матрас", "одеялц", "пледик", "кухонн"
    ],
    "товары для детей": [
        "памперс", "коляск", "игрушк", "детск", "слинг", "раскраск", "подгузн",
        "кроватк", "стульчик", "горшок", "качел", "машинк", "мобил"
    ],
    "украшения и аксессуары": [
        "кольц", "браслет", "цепочк", "серьг", "сумк", "ремень", "часы", "аксессуар",
        "брошь", "заколк", "подвеск", "ободок", "значок"
    ],
    "электроника": [
        "телефон", "смартфон", "наушник", "зарядк", "планшет", "ноутбук", "камера", "колонк",
        "гарнитур", "монитор", "проектор", "роутер", "мышь", "клавиатур", "тв"
    ],
    "нет товара": [
        "доставка", "упаковк", "продавец", "сервис", "возврат", "обмен", "спор", "курьер", "магазин",
        "оплата", "счёт", "заказ", "доставк", "рекламац"
    ]
}

def keyword_label(text):
  t = text.lower()
  counts = defaultdict(int)
  for cat, keys in kw.items():
    for k in keys:
      if k in t:
        counts[cat] += 1
  if not counts:
    return None, 0.0
  best = max(counts.items(), key=lambda x: x[1])
  values = sorted(counts.values(), reverse=True)
  if len(values) > 1 and values[0] == values[1]:
    return None, 0.0
  return best[0], 0.95

In [ ]:
df_train[['pred_category', 'pred_confidence']] = df_train['text_clean'].apply(lambda x: pd.Series(keyword_label(x)))

In [ ]:
df_train.head(25)

,text,_len_words,_len_chars,text_clean,pred_category,pred_confidence
0,"Заказали 14.10.2017 , получили 25.10.2017 \r\n...",31,205,"Заказали 14.10.2017 , получили 25.10.2017 На м...",NaN,0.00
1,"футболка хорошего качества,но футболка не как ...",34,203,"футболка хорошего качества,но футболка не как ...",NaN,0.00
2,Все отлично!!!,2,14,Все отлично!!!,NaN,0.00
3,"Рисунок не очень чёткий, а ткань прозрачная, в...",10,63,"Рисунок не очень чёткий, а ткань прозрачная, в...",NaN,0.00
4,плохо!!!Низ рваный..деньги не вернули!Открыла ...,9,75,плохо!!!Низ рваный..деньги не вернули!Открыла ...,нет товара,0.95
5,"обычная майка с ужасным запахом, есть косяки н...",12,72,"обычная майка с ужасным запахом, есть косяки н...",одежда,0.95
6,"Все как на фото, и цвет и форма.",8,32,"Все как на фото, и цвет и форма.",NaN,0.00
7,"Но рост 180 по колено,качество хорошее,доставк...",7,55,"Но рост 180 по колено,качество хорошее,доставк...",нет товара,0.95
8,Не подошло по размеру. Ткань тоже не понравилась,8,48,Не подошло по размеру. Ткань тоже не понравилась,NaN,0.00
9,"очень крутой топ,заказала 11.11 пришел 28.11 е...",10,64,"очень крутой топ,заказала 11.11 пришел 28.11 е...",нет товара,0.95


In [ ]:
df_train['pred_category'].value_counts()

pred_category
нет товара                494
электроника               234
одежда                    190
украшения и аксессуары      2
посуда                      2
текстиль                    1
бытовая техника             1
Name: count, dtype: int64

In [ ]:
df_train['pred_category'].isna().sum()

894

In [ ]:
df_train['pred_category'].unique()

array([nan, 'нет товара', 'одежда', 'электроника',
       'украшения и аксессуары', 'текстиль', 'посуда', 'бытовая техника'],
      dtype=object)

# Парсинг, промт - Авторазметка LLM

In [ ]:
categories = [
  {"id": 0, "name": "бытовая техника", "desc": "Пылесосы, чайники, утюги, микроволновки и т.п."},
  {"id": 1, "name": "обувь", "desc": "Кроссовки, ботинки, туфли, сандалии и т.п."},
  {"id": 2, "name": "одежда", "desc": "Футболки, платья, брюки, свитеры и т.п."},
  {"id": 3, "name": "посуда", "desc": "Тарелки, кастрюли, чашки, столовые приборы и т.п."},
  {"id": 4, "name": "текстиль", "desc": "Постельное белье, полотенца, пледы, шторы и т.п."},
  {"id": 5, "name": "товары для детей", "desc": "Игрушки, детская одежда, коляски, памперсы и т.п."},
  {"id": 6, "name": "украшения и аксессуары", "desc": "Кольца, браслеты, сумки, ремни, шарфы и т.п."},
  {"id": 7, "name": "электроника", "desc": "Телефоны, наушники, зарядные, ноутбуки, планшеты и т.п."},
  {"id": 8, "name": "нет товара", "desc": "Отзывы про доставку, упаковку, сервис, возврат или без упоминания конкретного товара."}
]

os.makedirs("configs", exist_ok=True)
with open("configs/categories.json", "w", encoding="utf-8") as f:
    json.dump({"categories": categories}, f, ensure_ascii=False, indent=2)

print("Saved configs/categories.json")

Saved configs/categories.json


In [ ]:
MODEL_NAME = "Qwen/Qwen2-7B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)

id_name_list = "\n".join([f"{c['id']} — {c['name']}: {c['desc']}" for c in categories])

SYSTEM = """Ты ассистент по классификации отзывов.
Твоя задача — отнести отзыв к одной из товарных категорий.
Если отзыв не содержит описания конкретного товара, а говорит только про доставку, оплату, возврат или сервис — тогда выбери категорию "нет товара".
Всегда отвечай строго JSON: {"label": <id категории>, "confidence": <0-100>}"""

USER_TEMPLATE = """
Категории:
{id_name_list}

Отзыв:
"{review}"

Требования:
- Выбери ровно один id категории из списка.
- Если отзыв описывает товар, всегда выбери соответствующую товарную категорию.
- "нет товара" использовать только если НЕТ описания товара.
- Ответ только JSON.
"""

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [ ]:
def extract_json(text):
    try:
        matches = re.findall(r"\{.*?\}", text, re.S)
        if not matches:
            return None
        snippet = matches[-1]  # последний блок в {} обычно правильный
        return json.loads(snippet)
    except Exception:
        return None

bad_cases = []

def run_llm(review, n_votes=1, temperature=0.3, top_p=0.9, max_new_tokens=64):
    prompt = USER_TEMPLATE.format(id_name_list=id_name_list, review=review)
    messages = [
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": prompt}
    ]
    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)

    results = []
    for _ in range(n_votes):
        out = model.generate(
            input_ids=input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id
        )
        decoded = tokenizer.decode(out[0], skip_special_tokens=True)
        parsed = extract_json(decoded)
        if parsed:
            results.append(parsed)
        else:
            bad_cases.append({"review": review, "decoded": decoded})

    if not results:
        return {"label": 8, "confidence": 20.0, "votes": {}, "raw": []}

    labels = [r.get("label", 8) for r in results]
    confs = [r.get("confidence", 50) for r in results]

    c = Counter(labels)
    best_label, best_count = c.most_common(1)[0]

    vote_fraction = best_count / len(labels)
    avg_conf = sum(confs) / len(confs)

    confidence = max(20.0, 0.6 * vote_fraction * 100 + 0.4 * avg_conf)

    return {"label": best_label, "confidence": confidence, "votes": dict(c), "raw": results}

In [ ]:
def keyword_label(text, kw_dict):
    t = text.lower()
    counts = Counter()
    for cat, keys in kw_dict.items():
        for k in keys:
            if k in t:
                counts[cat] += 1
    if not counts:
        return None, 0.0
    best_cat, best_score = counts.most_common(1)[0]
    return best_cat, 95.0  # keyword считаем "почти железным"

name2id = {c["name"]: c["id"] for c in categories}

def ensemble_label(text, kw_dict, tau_llm=75.0, kw_conf=95.0, n_votes=3):
    label_kw, conf_kw = keyword_label(text, kw_dict)
    id_kw = name2id.get(label_kw) if label_kw else None

    # Решаем, звать ли LLM
    need_llm = (id_kw is None) or (label_kw == "нет товара")
    llm_res = run_llm(text, n_votes=n_votes) if need_llm else None
    id_llm = llm_res.get("label") if llm_res else None
    conf_llm = llm_res.get("confidence", 0.0) if llm_res else 0.0

    # keyword даёт товарный класс (не "нет товара")
    if id_kw is not None and label_kw != "нет товара":
        if id_llm is not None and id_llm != id_kw:
            return {
                "label": id_llm,
                "confidence": conf_llm,
                "method": "conflict",
                "label_kw": id_kw,
                "label_llm": id_llm,
                "conf_kw": conf_kw,
                "conf_llm": conf_llm
            }
        return {
            "label": id_kw,
            "confidence": kw_conf,
            "method": "keyword",
            "label_kw": id_kw,
            "conf_kw": conf_kw
        }

    # keyword == "нет товара"
    if id_kw is not None and label_kw == "нет товара":
        if id_llm is not None:
            if conf_llm >= tau_llm:
                return {
                    "label": id_llm,
                    "confidence": conf_llm,
                    "method": "llm_confirm_from_kw_noitem",
                    "label_kw": id_kw,
                    "label_llm": id_llm,
                    "conf_llm": conf_llm
                }
            else:
                return {
                    "label": id_kw,
                    "confidence": conf_llm,
                    "method": "kw_noitem_lowconf",
                    "label_kw": id_kw,
                    "label_llm": id_llm,
                    "conf_llm": conf_llm
                }
        else:
            return {
                "label": id_kw,
                "confidence": 50.0,
                "method": "kw_noitem_lowconf",
                "label_kw": id_kw
            }

    # keyword отсутствует
    if id_kw is None:
        if id_llm is not None:
            if conf_llm >= tau_llm:
                return {
                    "label": id_llm,
                    "confidence": conf_llm,
                    "method": "llm",
                    "label_llm": id_llm,
                    "conf_llm": conf_llm
                }
            else:
                return {
                    "label": id_llm,
                    "confidence": conf_llm,
                    "method": "llm_lowconf",
                    "label_llm": id_llm,
                    "conf_llm": conf_llm
                }
        else:
            return {
                "label": None,
                "confidence": 0.0,
                "method": "noinfo"
            }

    return {"label": None, "confidence": 0.0, "method": "fallback"}

Проверка на 10 примерах
Подебажим

In [ ]:
for i in range(5):
    review = df_train.loc[i, "text_clean"]
    res = run_llm(review, n_votes=1)  # 1 для быстроты
    print(i, review[:60], "=>", res)

0 Заказали 14.10.2017 , получили 25.10.2017 На мой размер 42,  => {'label': 7, 'confidence': 98.0, 'votes': {7: 1}, 'raw': [{'label': 7, 'confidence': 95}]}
1 футболка хорошего качества,но футболка не как для девушек и  => {'label': 2, 'confidence': 98.0, 'votes': {2: 1}, 'raw': [{'label': 2, 'confidence': 95}]}
2 Все отлично!!! => {'label': 8, 'confidence': 20.0, 'votes': {}, 'raw': []}
3 Рисунок не очень чёткий, а ткань прозрачная, видно нижнее бе => {'label': 4, 'confidence': 98.0, 'votes': {4: 1}, 'raw': [{'label': 4, 'confidence': 95}]}
4 плохо!!!Низ рваный..деньги не вернули!Открыла спор,но его по => {'label': 5, 'confidence': 98.0, 'votes': {5: 1}, 'raw': [{'label': 5, 'confidence': 95}]}


Итоговый прогон

In [ ]:
os.makedirs("outputs", exist_ok=True)

results = []
for idx, row in tqdm(df_train.reset_index(drop=True).iterrows(), total=len(df_train)):
    res = run_llm(row["text_clean"], n_votes=1)  # пока 1 для экономии времени
    res.update({
        "idx": idx,
        "text_clean": row["text_clean"],
        "len_words": row.get("_len_words"),
        "len_chars": row.get("_len_chars"),
    })
    results.append(res)

pseudo_df = pd.DataFrame(results)

# 🔧 Сериализуем сложные поля, чтобы parquet не падал
def serialize(x):
    if isinstance(x, (dict, list)):
        return json.dumps(x, ensure_ascii=False)
    return x

for col in ["votes", "raw"]:
    if col in pseudo_df.columns:
        pseudo_df[col] = pseudo_df[col].apply(serialize)

# ✅ Теперь сохраняем
pseudo_df.to_parquet("outputs/pseudo_labels.parquet", index=False)
print("Сохранили pseudo_df:", pseudo_df.shape)

print("Плохих примеров (не JSON):", len(bad_cases))

print("\nРаспределение по классам:")
print(pseudo_df["label"].value_counts())

print("\nСредняя уверенность:")
print(pseudo_df["confidence"].mean())

print("\nПримеры низкоуверенных:")
print(pseudo_df[pseudo_df["confidence"] < 75].head(5)[["text_clean", "label", "confidence"]])

  0%|          | 0/1818 [00:00<?, ?it/s]

Сохранили pseudo_df: (1818, 8)
Плохих примеров (не JSON): 330

Распределение по классам:
label
2    817
8    501
4    129
5    127
7    125
3     70
0     45
6      4
Name: count, dtype: int64

Средняя уверенность:
91.57315731573158

Примеры низкоуверенных:
                                           text_clean  label  confidence
4   плохо!!!Низ рваный..деньги не вернули!Открыла ...      8        20.0
44  пришла за 23 дня.качество прекрасное.ткань пох...      8        20.0
53  мало того, что заказ отправили не туда. так ещ...      8        20.0
57                 Самый долгий заказ на Алиэкспресс.      8        20.0
77  Шло в Москву очень долго, доставка хреновая. О...      8        20.0


In [ ]:
pseudo_df.head(10)

,label,confidence,votes,raw,idx,text_clean,len_words,len_chars
0,7,98.0,"{""7"": 1}","[{""label"": 7, ""confidence"": 95}]",0,"Заказали 14.10.2017 , получили 25.10.2017 На м...",31,205
1,2,98.0,"{""2"": 1}","[{""label"": 2, ""confidence"": 95}]",1,"футболка хорошего качества,но футболка не как ...",34,203
2,8,100.0,"{""8"": 1}","[{""label"": 8, ""confidence"": 100}]",2,Все отлично!!!,2,14
3,4,98.0,"{""4"": 1}","[{""label"": 4, ""confidence"": 95}]",3,"Рисунок не очень чёткий, а ткань прозрачная, в...",10,63
4,8,20.0,{},[],4,плохо!!!Низ рваный..деньги не вернули!Открыла ...,9,75
5,2,98.0,"{""2"": 1}","[{""label"": 2, ""confidence"": 95}]",5,"обычная майка с ужасным запахом, есть косяки н...",12,72
6,0,100.0,"{""0"": 1}","[{""label"": 0, ""confidence"": 100}]",6,"Все как на фото, и цвет и форма.",8,32
7,2,98.0,"{""2"": 1}","[{""label"": 2, ""confidence"": 95}]",7,"Но рост 180 по колено,качество хорошее,доставк...",7,55
8,4,94.0,"{""4"": 1}","[{""label"": 4, ""confidence"": 85}]",8,Не подошло по размеру. Ткань тоже не понравилась,8,48
9,2,100.0,"{""2"": 1}","[{""label"": 2, ""confidence"": 100}]",9,"очень крутой топ,заказала 11.11 пришел 28.11 е...",10,64


ДАЛЕЕ КОД БЫСМЫСЛЕННО ПОКА СМОТРЕТЬ

In [ ]:
df = pd.DataFrame(pseudo_df)

# Сохраняем в CSV
df.to_csv('pseudo_df.csv', index=False, encoding='utf-8')

from IPython.display import FileLink
FileLink('pseudo_df.csv')

/kaggle/working/pseudo_df.csv

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

In [3]:
pseudo_df = pd.read_csv('/kaggle/input/tbank-sirius-continue/pseudo_df.csv')

In [4]:
pseudo_df.head(5)

,label,confidence,votes,raw,idx,text_clean,len_words,len_chars
0,7,98.0,"{""7"": 1}","[{""label"": 7, ""confidence"": 95}]",0,"Заказали 14.10.2017 , получили 25.10.2017 На м...",31,205
1,2,98.0,"{""2"": 1}","[{""label"": 2, ""confidence"": 95}]",1,"футболка хорошего качества,но футболка не как ...",34,203
2,8,100.0,"{""8"": 1}","[{""label"": 8, ""confidence"": 100}]",2,Все отлично!!!,2,14
3,4,98.0,"{""4"": 1}","[{""label"": 4, ""confidence"": 95}]",3,"Рисунок не очень чёткий, а ткань прозрачная, в...",10,63
4,8,20.0,{},[],4,плохо!!!Низ рваный..деньги не вернули!Открыла ...,9,75


In [5]:
pseudo_df.shape

(1818, 8)

In [6]:
filter_df = pseudo_df[pseudo_df["confidence"] >= 75].copy()
print("Оставили после фильтрации (conf >= 75):", filter_df.shape)

Оставили после фильтрации (conf >= 75): (1656, 8)


In [7]:
filter_df = filter_df[["text_clean", "label", "confidence"]].reset_index(drop=True)

train_df, val_df = train_test_split(
    filter_df,
    test_size=0.2,
    random_state=42,
    stratify=filter_df["label"]
)

print("Train:", train_df.shape, "\nVal:", val_df.shape)

Train: (1324, 3) 
Val: (332, 3)


In [8]:
print("\nРаспределение по классам (train):")
print(train_df["label"].value_counts())

print("\nРаспределение по классам (val):")
print(val_df["label"].value_counts())

print("\nСредняя уверенность (train):", train_df["confidence"].mean())
print("Средняя уверенность (val):", val_df["confidence"].mean())


Распределение по классам (train):
label
2    653
8    271
4    103
5    102
7    100
3     56
0     36
6      3
Name: count, dtype: int64

Распределение по классам (val):
label
2    164
8     68
4     26
5     25
7     25
3     14
0      9
6      1
Name: count, dtype: int64

Средняя уверенность (train): 98.61027190332327
Средняя уверенность (val): 98.43373493975903


In [9]:
MODEL_NAME = "cointegrated/rubert-tiny2"
NUM_LABELS = train_df["label"].nunique()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [10]:
train_df.head(5)

,text_clean,label,confidence
1311,"Время покупателя закончилось, деньги не вернул...",8,100.0
432,Доставка 15 дней. Заказываю уже второй комплек...,8,100.0
48,"Очень милая и приятная на ощупь, пришла очень ...",4,98.0
1177,Если честно товаром совсем недовольна. Кружево...,3,98.0
1036,"очень широкая майка, как наволочка",2,98.0


In [11]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_df['label'] = le.fit_transform(train_df['label'])
val_df['label'] = le.transform(val_df['label'])

NUM_LABELS = len(le.classes_)
print("NUM_LABELS:", NUM_LABELS)
print("train labels min/max:", train_df['label'].min(), train_df['label'].max())

NUM_LABELS: 8
train labels min/max: 0 7


In [12]:
def preprocess(batch):
  return tokenizer(
      batch["text_clean"],
      truncation=True,
      padding="max_length",
      max_length=128
  )

train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)

train_ds = train_ds.map(preprocess, batched=True)
val_ds = val_ds.map(preprocess, batched=True)

train_ds = train_ds.rename_column("label", "labels").with_format("torch")
val_ds = val_ds.rename_column("label", "labels").with_format("torch")

Map:   0%|          | 0/1324 [00:00<?, ? examples/s]

Map:   0%|          | 0/332 [00:00<?, ? examples/s]

In [17]:
classes = np.arange(NUM_LABELS)  # обязательно соответствие NUM_LABELS
weights = compute_class_weight(
    class_weight="balanced",
    classes=classes,
    y=train_df["label"]
)
class_weights = torch.tensor(weights, dtype=torch.float).to("cuda")
print("class_weights:", class_weights)

class_weights: tensor([ 4.5972,  0.2534,  2.9554,  1.6068,  1.6225, 55.1667,  1.6550,  0.6107],
       device='cuda:0')


In [18]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS
)

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [37]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    # Ваши обязательные метрики (добавьте нужные)
    accuracy = accuracy_score(labels, predictions)
    f1_weighted = f1_score(labels, predictions, average='weighted')
    f1_macro = f1_score(labels, predictions, average='macro')
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    
    return {
        'accuracy': accuracy,
        'f1_weighted': f1_weighted,
        'f1_macro': f1_macro,
        'precision': precision,
        'recall': recall
    }

In [45]:
training_args = TrainingArguments(
    output_dir="outputs/model_4",
    eval_strategy="epoch",  # было: evaluation_strategy
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=1.5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_weighted",  #  метрика
    greater_is_better=True,  # важно указать для метрик
    push_to_hub=False,
    report_to="none",  # чтобы не логировать в сторонние сервисы
    logging_dir='./logs',  # директория для логов
    logging_steps=100,  # частоту логирования можно настроить
)

In [39]:
from transformers import Trainer
from torch.nn import CrossEntropyLoss

In [46]:
def custom_loss(model, inputs, return_outputs=False, **kwargs):
    """
    Кастомный loss для Trainer с учетом class_weights.
    **kwargs ловит любые дополнительные аргументы Trainer.
    """
    labels = inputs.pop("labels")
    outputs = model(**inputs)
    logits = outputs.logits
    loss_fct = CrossEntropyLoss(weight=class_weights.to(logits.device))
    loss = loss_fct(logits.view(-1, NUM_LABELS), labels.view(-1))
    return (loss, outputs) if return_outputs else loss

In [47]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_36/3457493100.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [48]:
trainer.compute_loss = custom_loss

In [49]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Weighted,F1 Macro,Precision,Recall
1,1.462200,1.569004,0.572289,0.574550,0.337638,0.612698,0.572289
2,1.397300,1.523839,0.560241,0.568683,0.327555,0.610110,0.560241
3,1.327400,1.489724,0.596386,0.603590,0.344467,0.632879,0.596386
4,1.260200,1.452577,0.590361,0.601725,0.341809,0.634858,0.590361
5,1.209700,1.427468,0.605422,0.618197,0.366474,0.651105,0.605422
6,1.152600,1.411129,0.608434,0.621261,0.368035,0.652950,0.608434
7,1.120000,1.392007,0.623494,0.634602,0.377530,0.662432,0.623494
8,1.063200,1.387715,0.632530,0.642428,0.382816,0.665190,0.632530
9,1.036900,1.381619,0.632530,0.640108,0.379851,0.658991,0.632530
10,1.007200,1.364972,0.620482,0.632870,0.383860,0.659054,0.620482


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

TrainOutput(global_step=1245, training_loss=1.1099879927424543, metrics={'train_runtime': 42.1403, 'train_samples_per_second': 471.283, 'train_steps_per_second': 29.544, 'total_flos': 36641581793280.0, 'train_loss': 1.1099879927424543, 'epoch': 15.0})